## Gathered Notebook

This notebook was generated by the Gather Extension. The intent is that it contains only the code and cells required to produce the same results as the cell originally selected for gathering. Please note that the Python analysis is quite conservative, so if it is unsure whether a line of code is necessary for execution, it will err on the side of including it.

**Please let us know if you are satisfied with what was gathered [here](https://aka.ms/gatherfeedback).**

Thanks

In [ ]:
def get_categories(df,max_unique=0,num=False):
    categories=[]
    numerics=[]
    for col in df.columns:
        if str(df[col].dtypes)=="category":
                categories.append(col)
        elif df[col].nunique()<=max_unique and str(df[col].dtypes)!="datetime64[ns]":
            categories.append(col)
        elif str(df[col].dtypes)!="datetime64[ns]":
            numerics.append(col)
    if num:
        return numerics
    else:
        return categories
def corr_matrix(df):# return la matrice de correlation en prenant en consideration le test chi-deu (varibale categorique - categorique) et le test anova (variable numerique- categorique)
    corr=df.corr()
    for cat_i in get_categories(df=df):
        for cat_j in get_categories(df=df):
            if cat_i!=cat_j:
                cont = df[[cat_i, cat_j]].pivot_table(index=cat_i, columns=cat_j, aggfunc=len).fillna(0).copy()
                st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)
                corr.loc[cat_i,cat_j]=round(st_p/0.1,5) if st_p/0.1 <=1 else 1
            else:
                corr.loc[cat_i,cat_j]=1
    for cat_i in get_categories(df=df,num=True):
        for cat_j in get_categories(df=df):
            string=cat_i+' ~ C('+cat_j+')'
            model = ols(string, data=df).fit()
            aov_table = sm.stats.anova_lm(model, typ=2)
            corr.loc[cat_i,cat_j]=round(aov_table.loc["C("+cat_j+")","PR(>F)"]/0.1,5) if aov_table.loc["C("+cat_j+")","PR(>F)"]/0.1 <=1 else 1
            corr.loc[cat_j,cat_i]=round(aov_table.loc["C("+cat_j+")","PR(>F)"]/0.1,5) if aov_table.loc["C("+cat_j+")","PR(>F)"]/0.1 <=1 else 1
    return corr
def heatmap(df):#affichage du heatmap de la matrice de correlation
    df_corr=corr_matrix(df)
    fig, ax = plt.subplots(figsize=(12, 10))
    # mask
    mask = np.triu(np.ones_like(df_corr, dtype=bool))
    # adjust mask and df
    mask = mask[1:, :-1]
    corr = df_corr.iloc[1:,:-1].copy()
    cmap = sb.diverging_palette(0, 230, 90, 60, as_cmap=True)
    sb.heatmap(corr, mask=mask, annot=True, fmt=".2f", 
            linewidths=5, cmap=cmap,  
            cbar_kws={"shrink": .8}, square=True)
    yticks = [i.upper() for i in corr.index]
    xticks = [i.upper() for i in corr.columns]
    plt.yticks(plt.yticks()[0], labels=yticks, rotation=0)
    plt.xticks(plt.xticks()[0], labels=xticks)
    title = 'CORRELATION MATRIX\nSAMPLED CEREALS COMPOSITION\n'
    plt.title(title, loc='left', fontsize=18)
    plt.show()
    df_corr=corr_matrix(df)